# Mesh decimation using Google Colab
* For decimating large meshes (e.g., astrocytes), you will need to process using high-memory resource setting  
* On a laptop system, astrocyte decimation takes several hours (up to 12), whereas the same mesh will take less than 10 min on Google Colab  
* Requires a subscription to Google Colab Pro
* Requires the mesh h5 files to be located in your connected Google Drive (see code below)

In [ ]:
pip install pyvista

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 MB 17.5 MB/s eta 0:00:00


In [ ]:
# Access your Google Drive (where your data file resides)
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
print(os.listdir('gdrive/MyDrive/Colab Notebooks/data/c3_cell_meshes/'))

['33487935587_0.h5', 'dec']


In [ ]:
# Runtime > Change runtime type > Hardware accelerator: None > Runtime shape: High-RAM

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import h5py
import pyvista as pv

# Set up paths and parameters
obj_path = 'gdrive/MyDrive/Colab Notebooks/data/c3_cell_meshes/'
dec_path = 'gdrive/MyDrive/Colab Notebooks/data/c3_cell_meshes/dec/'
dec_prcnt = 95

# Load cell cellids into a list
cell_list = [33487935587]

def read_h5_file(file_path):
    with h5py.File(file_path, 'r') as f:
        vertices = np.array(f['vertices'])
        faces = np.array(f['faces'])
    return vertices, faces

def create_pyvista_mesh(vertices, faces):
    # PyVista expects faces to be in the format (n, v1, v2, v3, ...)
    faces_with_size = np.hstack([np.full((faces.shape[0], 1), faces.shape[1]), faces])
    return pv.PolyData(vertices, faces_with_size)

def process_obj(cellid, dec_prcnt, obj_path, dec_path):
    start_time = time.time()

    # Create the full path to the file
    file_path = os.path.join(obj_path, str(cellid) + '_0.h5')
    print(f"Loading {file_path}")

    # Load the file
    vertices, faces = read_h5_file(file_path)
    load_time = time.time()
    print(f"Loaded {file_path} in {load_time - start_time:.2f} seconds")

    # Create PyVista mesh
    mesh = create_pyvista_mesh(vertices, faces)
    print(f"Original mesh info: {mesh}")

    # Perform decimation
    print(f"Decimating {file_path} by {dec_prcnt}%")
    simplified_mesh = mesh.decimate(target_reduction=dec_prcnt / 100)  # Reduce to x% of the original mesh
    decimate_time = time.time()
    print(f"Decimated {file_path} in {decimate_time - load_time:.2f} seconds")

    # Print simplified mesh information
    print(f"Simplified mesh info: {simplified_mesh}")

    # Ensure the output directory exists
    if not os.path.exists(dec_path):
        os.makedirs(dec_path)

    # Save the simplified mesh in PLY format
    output_filename = os.path.basename(file_path).replace('.h5', '.ply')
    output_path = os.path.join(dec_path, output_filename)

    try:
        simplified_mesh.save(output_path)
        save_time = time.time()
        print(f"Saved {output_path} in {save_time - decimate_time:.2f} seconds")
    except Exception as e:
        print(f"Error saving {output_path}: {e}")

    total_time = time.time() - start_time
    print(f"Total time: {total_time // 60} minutes and {total_time % 60:.2f} seconds")

# Process each cellid in the cell_list
for cellid in cell_list:
    process_obj(cellid, dec_prcnt, obj_path, dec_path)


Loading gdrive/MyDrive/Colab Notebooks/data/c3_cell_meshes/33487935587_0.h5
Loaded gdrive/MyDrive/Colab Notebooks/data/c3_cell_meshes/33487935587_0.h5 in 9.86 seconds
Original mesh info: PolyData (0x7e6b16136da0)
  N Cells:    50726849
  N Points:   25157643
  N Strips:   0
  X Bounds:   1.921e+06, 2.020e+06
  Y Bounds:   1.856e+06, 1.993e+06
  Z Bounds:   5.923e+04, 1.746e+05
  N Arrays:   0
Decimating gdrive/MyDrive/Colab Notebooks/data/c3_cell_meshes/33487935587_0.h5 by 95%
Decimated gdrive/MyDrive/Colab Notebooks/data/c3_cell_meshes/33487935587_0.h5 in 485.79 seconds
Simplified mesh info: PolyData (0x7e6b55d0efe0)
  N Cells:    2536342
  N Points:   1267026
  N Strips:   0
  X Bounds:   1.921e+06, 2.020e+06
  Y Bounds:   1.856e+06, 1.992e+06
  Z Bounds:   5.924e+04, 1.746e+05
  N Arrays:   0
Saved gdrive/MyDrive/Colab Notebooks/data/c3_cell_meshes/dec/33487935587_0.ply in 2.59 seconds
Total time: 8.0 minutes and 18.23 seconds
